In [1]:
from langchain.document_loaders import PyMuPDFLoader


In [2]:
pdf_path = r"C:\Users\musta\OneDrive\Desktop\hybrid search\data\Searching_for_Best_Practices_in_Retrieval-Augmente.pdf"

In [3]:
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()

In [4]:
for doc in documents:
    print(doc.page_content)

Searching for Best Practices in Retrieval-Augmented
Generation
Xiaohua Wang, Zhenghua Wang, Xuan Gao, Feiran Zhang,
Yixin Wu, Zhibo Xu, Tianyuan Shi, Zhengyuan Wang, Shizheng Li,
Qi Qian, Ruicheng Yin, Changze Lv, Xiaoqing Zheng∗
, Xuanjing Huang
School of Computer Science, Fudan University, Shanghai, China
Shanghai Key Laboratory of Intelligent Information Processing
{xiaohuawang22,zhenghuawang23}@m.fudan.edu.cn
{zhengxq,xjhuang}@fudan.edu.cn
Abstract
Retrieval-augmented generation (RAG) techniques have proven to be effective
in integrating up-to-date information, mitigating hallucinations, and enhancing
response quality, particularly in specialized domains. While many RAG approaches
have been proposed to enhance large language models through query-dependent
retrievals, these approaches still suffer from their complex implementation and
prolonged response times. Typically, a RAG workflow involves multiple processing
steps, each of which can be executed in various ways. Here, we invest

In [5]:
for doc in documents:
    doc.page_content = doc.page_content.lower()

for doc in documents:
    print(doc.page_content)


searching for best practices in retrieval-augmented
generation
xiaohua wang, zhenghua wang, xuan gao, feiran zhang,
yixin wu, zhibo xu, tianyuan shi, zhengyuan wang, shizheng li,
qi qian, ruicheng yin, changze lv, xiaoqing zheng∗
, xuanjing huang
school of computer science, fudan university, shanghai, china
shanghai key laboratory of intelligent information processing
{xiaohuawang22,zhenghuawang23}@m.fudan.edu.cn
{zhengxq,xjhuang}@fudan.edu.cn
abstract
retrieval-augmented generation (rag) techniques have proven to be effective
in integrating up-to-date information, mitigating hallucinations, and enhancing
response quality, particularly in specialized domains. while many rag approaches
have been proposed to enhance large language models through query-dependent
retrievals, these approaches still suffer from their complex implementation and
prolonged response times. typically, a rag workflow involves multiple processing
steps, each of which can be executed in various ways. here, we invest

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [7]:

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  
    chunk_overlap=100 
)


In [8]:
chunks = []
for doc in documents:
    text_chunks = splitter.split_text(doc.page_content)
    lower_chunks = [chunk.lower() for chunk in text_chunks]
    chunks.extend(lower_chunks)

for chunk in chunks:
    print(chunk)

searching for best practices in retrieval-augmented
generation
xiaohua wang, zhenghua wang, xuan gao, feiran zhang,
yixin wu, zhibo xu, tianyuan shi, zhengyuan wang, shizheng li,
qi qian, ruicheng yin, changze lv, xiaoqing zheng∗
, xuanjing huang
school of computer science, fudan university, shanghai, china
shanghai key laboratory of intelligent information processing
{xiaohuawang22,zhenghuawang23}@m.fudan.edu.cn
{zhengxq,xjhuang}@fudan.edu.cn
abstract
{xiaohuawang22,zhenghuawang23}@m.fudan.edu.cn
{zhengxq,xjhuang}@fudan.edu.cn
abstract
retrieval-augmented generation (rag) techniques have proven to be effective
in integrating up-to-date information, mitigating hallucinations, and enhancing
response quality, particularly in specialized domains. while many rag approaches
have been proposed to enhance large language models through query-dependent
retrievals, these approaches still suffer from their complex implementation and
retrievals, these approaches still suffer from their complex imp

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [10]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Bot

In [11]:
from langchain.docstore.document import Document
docs = [Document(page_content=chunk) for chunk in chunks]

In [12]:
db2 = Chroma.from_documents(docs, embedding=embedding, persist_directory="./chroma_db")


In [13]:
db3 = Chroma(persist_directory="./chroma_db", embedding_function=embedding)

c:\Users\musta\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [14]:
retriever = db3.as_retriever(search_kwargs={"k": 3})

In [15]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever


In [16]:
keyword = BM25Retriever.from_documents(docs)


In [17]:
keyword.k = 3

In [49]:
keyword

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001C00BD91450>, k=3)

In [18]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[keyword, retriever],
    retriever_weights=[0.5, 0.5] 
)

In [19]:
from langchain_community.llms import Ollama


In [20]:
llm = Ollama(model="llama3")


In [21]:
from langchain.chains import RetrievalQA


In [22]:
# hybrid_chain = RetrievalQA.from_chain_type(
#     llm, retriever=ensemble_retriever
# )

In [23]:
# hybrid_chain.invoke("what is the Best Practices for Implementing RAG?")

{'query': 'what is the Best Practices for Implementing RAG?',
 'result': 'According to the text, there are two distinct recipes or practices for implementing RAG systems:\n\n1. **Best Performance Practice**: To achieve the highest performance, it is recommended to incorporate the query classification module and implement the hybrid method for retrieval.\n2. **Balanced Efficiency Practice**: To achieve a balance between performance and efficiency, it is recommended to incorporate the query classification module, use the hybrid method for retrieval, opt for TildeV2 for reranking, choose Reverse for repacking, and employ Recomp for summarization.\n\nThese practices are based on the experimental findings of this study.'}

In [28]:
COHERE_API_KEY = "Q1tNApphbMywrTviu1WfdEYa3DfNr8NtwhlGAiYh"

In [35]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [37]:

import getpass
import os


api_key = getpass.getpass("Enter your Cohere API key: ")
compressor = CohereRerank(cohere_api_key=api_key)


In [39]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

In [40]:
hybrid_chain = RetrievalQA.from_chain_type(
    llm, retriever=compression_retriever
)

In [48]:
hybrid_chain.invoke("explain that topic with easy words and additionally add example for better understand:Hybrid Search with Different Weight on Sparse Retrieval?")

{'query': 'explain that topic with easy words and additionally add example for better understand:Hybrid Search with Different Weight on Sparse Retrieval?',
 'result': 'Let me try to break it down in simpler terms:\n\n**What is Hybrid Search?**\n\nImagine you\'re searching for a book online. You have two ways to find the book: one way is to look at the book\'s title, author, and summary (like a "sparse" search), and another way is to analyze the book\'s contents, like its keywords and themes (like a "dense" search). Hybrid Search combines these two methods to get the best results.\n\n**What does "Different Weight on Sparse Retrieval" mean?**\n\nIn Hybrid Search, you can adjust how much each method contributes to the final result. The weight refers to how much more important one method is compared to the other. For example:\n\n* If you set a high weight for sparse retrieval (looking at titles and summaries), it means that method will be more influential in determining the search results.